### Setup

In [51]:
import pandas as pd

In [52]:
wb_df = pd.read_csv("P_Data_Extract_From_World_Development_Indicators/664d433e-5594-40be-a7a4-766171ee9f09_Data.csv")

plty_df = pd.read_excel("POLITY5-PRC.xlsx")

In [53]:
wb_df.head()

,Country Name,Country Code,Series Name,Series Code,1960 [YR1960],1961 [YR1961],1962 [YR1962],1963 [YR1963],1964 [YR1964],1965 [YR1965],...,2014 [YR2014],2015 [YR2015],2016 [YR2016],2017 [YR2017],2018 [YR2018],2019 [YR2019],2020 [YR2020],2021 [YR2021],2022 [YR2022],2023 [YR2023]
0,Afghanistan,AFG,Real interest rate (%),FR.INR.RINR,..,..,..,..,..,..,...,14.3516893380043,12.2525481629518,17.5839381624543,12.1411782956513,..,..,..,..,..,..
1,Afghanistan,AFG,CPIA debt policy rating (1=low to 6=high),IQ.CPA.DEBT.XQ,..,..,..,..,..,..,...,3,3,2.5,2.5,2.5,2.5,2.5,..,1.5,1.5
2,Afghanistan,AFG,"Unemployment, total (% of total labor force) (...",SL.UEM.TOTL.ZS,..,..,..,..,..,..,...,7.915,9.011,10.1,11.184,11.206,11.224,11.71,11.934,14.1,14.386
3,Afghanistan,AFG,"Inflation, GDP deflator (annual %)",NY.GDP.DEFL.KD.ZG,..,..,..,..,..,..,...,0.56694454253261,2.44756300147404,-2.19752646733461,2.40365618765746,2.07134857783487,6.52147984832531,6.9629462552564,2.83899625081041,9.40623933931322,2.9616535410169
4,Afghanistan,AFG,GDP growth (annual %),NY.GDP.MKTP.KD.ZG,..,..,..,..,..,..,...,2.72454336219565,1.45131466066431,2.26031420279821,2.6470032027451,1.18922812944517,3.91160341625552,-2.35110067203466,-20.7388393676343,-6.24017199240269,2.71088679128353


In [54]:
plty_df.head()

,Economy ISO3,Economy Name,Indicator ID,Indicator,Attribute 1,Attribute 2,Attribute 3,Partner,1776,1777,...,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
0,AFG,Afghanistan,POLITY5.PRC.autoc,Polity database: Institutionalized Autocracy I...,-,-,-,-,NaN,NaN,...,-66.0,-66.0,-66.0,2.0,2.0,2.0,2.0,2.0,NaN,NaN
1,AFG,Afghanistan,POLITY5.PRC.democ,Polity database: Institutionalized Democracy I...,-,-,-,-,NaN,NaN,...,-66.0,-66.0,-66.0,1.0,1.0,1.0,1.0,1.0,NaN,NaN
2,AFG,Afghanistan,POLITY5.PRC.durable,Polity database: Regime Durability Index,-,-,-,-,NaN,NaN,...,0.0,0.0,0.0,0.0,1.0,2.0,3.0,4.0,NaN,NaN
3,AFG,Afghanistan,POLITY5.PRC.exconst,Polity database: Executive Constraints Concept,-,-,-,-,NaN,NaN,...,-66.0,-66.0,-66.0,4.0,4.0,4.0,4.0,4.0,NaN,NaN
4,AFG,Afghanistan,POLITY5.PRC.exrec,Polity database: Executive Recruitment Concept,-,-,-,-,NaN,NaN,...,-66.0,-66.0,-66.0,3.0,3.0,3.0,3.0,3.0,NaN,NaN


In [55]:
# ----------------------
# Cleaning wb_df
# ----------------------
# Remove [YRxxxx] text from year columns
wb_df.columns = wb_df.columns.str.replace(r'\s*\[YR\d+\]', '', regex=True)

# Melt wb_df to long format
wb_long_df = wb_df.melt(
    id_vars=["Country Name", "Country Code", "Series Name", "Series Code"],
    var_name="Year", value_name="Value"
)

# Convert Year to int
wb_long_df["Year"] = pd.to_numeric(wb_long_df["Year"], errors='coerce', downcast='integer')
wb_long_df = wb_long_df.dropna(subset=["Year"])  # drop rows if Year couldn't be converted
wb_long_df["Year"] = wb_long_df["Year"].astype(int)

# Convert Value to numeric
wb_long_df["Value"] = pd.to_numeric(wb_long_df["Value"], errors='coerce')

# Pivot table using mean. If there are non-numeric entries, now they are NaN.
wb_wide_df = wb_long_df.pivot_table(
    index=["Country Code", "Year"],
    columns="Series Name",
    values="Value",
    aggfunc='mean'
).reset_index()

# Rename "Country Code" to "ISO3"
wb_wide_df = wb_wide_df.rename(columns={"Country Code": "ISO3"})

wb_wide_df = wb_wide_df[(wb_wide_df["Year"] >= 1980) & (wb_wide_df["Year"] <= 2020)]


In [56]:
# ----------------------
# Cleaning plty_df
# ----------------------
# Assume plty_df has columns:
# "Economy ISO3", "Economy Name", "Indicator ID", "Indicator", "Attribute 1", "Attribute 2", "Attribute 3", "Partner", and yearly columns like 1776, 1777, ...
plty_long_df = plty_df.melt(
    id_vars=["Economy ISO3", "Economy Name", "Indicator ID", "Indicator", "Attribute 1", "Attribute 2", "Attribute 3", "Partner"],
    var_name="Year", value_name="Indicator_Value"
)

# Rename "Economy ISO3" to "ISO3"
plty_long_df = plty_long_df.rename(columns={"Economy ISO3": "ISO3"})

# Ensure Year is integer
plty_long_df["Year"] = plty_long_df["Year"].astype(int)

# Pivot so each Indicator is a column
plty_pivot_df = plty_long_df.pivot_table(
    index=["ISO3", "Year"],
    columns="Indicator",
    values="Indicator_Value",
    aggfunc='mean'  # If duplicates occur for the same indicator, year, country
).reset_index()

plty_pivot_df = plty_pivot_df[(plty_pivot_df["Year"] >= 1980) & (plty_pivot_df["Year"] <= 2020)]

plty_pivot_df = plty_pivot_df[(plty_pivot_df["Polity database: Combined Polity Score"] >= -10) & (plty_pivot_df["Polity database: Combined Polity Score"] <= 10)]

In [57]:
plty_pivot_df

Indicator,ISO3,Year,Polity database: Combined Polity Score,Polity database: Competitiveness of Executive Recruitment,Polity database: Executive Constraints (Decision Rules),Polity database: Executive Constraints Concept,Polity database: Executive Recruitment Concept,Polity database: Institutionalized Autocracy Index,Polity database: Institutionalized Democracy Index,Polity database: Openness of Executive Recruitment,Polity database: Political Competition Concept,Polity database: Regime Durability Index,Polity database: Regulation of Chief Executive Recruitment,Polity database: Regulation of Participation,Polity database: Revised Combined Polity Score (p5),Polity database: The Competitiveness of Participation
189,AFG,1989,-8.0,1.0,2.0,2.0,3.0,8.0,0.0,4.0,1.0,0.0,2.0,4.0,-8.0,1.0
190,AFG,1990,-8.0,1.0,2.0,2.0,3.0,8.0,0.0,4.0,1.0,1.0,2.0,4.0,-8.0,1.0
191,AFG,1991,-8.0,1.0,2.0,2.0,3.0,8.0,0.0,4.0,1.0,2.0,2.0,4.0,-8.0,1.0
196,AFG,1996,-7.0,0.0,1.0,1.0,4.0,7.0,0.0,0.0,1.0,0.0,1.0,4.0,-7.0,1.0
197,AFG,1997,-7.0,0.0,1.0,1.0,4.0,7.0,0.0,0.0,1.0,1.0,1.0,4.0,-7.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16343,ZWE,2014,4.0,2.0,5.0,5.0,7.0,1.0,5.0,4.0,6.0,1.0,2.0,3.0,4.0,3.0
16344,ZWE,2015,4.0,2.0,5.0,5.0,7.0,1.0,5.0,4.0,6.0,2.0,2.0,3.0,4.0,3.0
16345,ZWE,2016,4.0,2.0,5.0,5.0,7.0,1.0,5.0,4.0,6.0,3.0,2.0,3.0,4.0,3.0
16346,ZWE,2017,4.0,2.0,5.0,5.0,7.0,1.0,5.0,4.0,6.0,4.0,2.0,3.0,4.0,3.0


In [58]:
# ----------------------
# Merge wb_wide_df and plty_pivot_df
# ----------------------
merged_df = pd.merge(wb_wide_df, plty_pivot_df, on=["ISO3", "Year"], how="inner")

# Sort for groupby operations
merged_df = merged_df.sort_values(by=["ISO3", "Year"])

# Create regime_change_next_year variable
merged_df["regime_change_next_year"] = (
    merged_df.groupby("ISO3")["Polity database: Regime Durability Index"]
    .transform(lambda x: (x.shift(-1) < x).astype(int))
)

In [61]:
merged_df.head()

,ISO3,Year,CPIA debt policy rating (1=low to 6=high),"Central government debt, total (% of GDP)",GDP growth (annual %),"Inflation, GDP deflator (annual %)","Labor force participation rate, total (% of total population ages 15-64) (modeled ILO estimate)",Real effective exchange rate index (2010 = 100),Real interest rate (%),"Unemployment, total (% of total labor force) (modeled ILO estimate)",...,Polity database: Institutionalized Autocracy Index,Polity database: Institutionalized Democracy Index,Polity database: Openness of Executive Recruitment,Polity database: Political Competition Concept,Polity database: Regime Durability Index,Polity database: Regulation of Chief Executive Recruitment,Polity database: Regulation of Participation,Polity database: Revised Combined Polity Score (p5),Polity database: The Competitiveness of Participation,regime_change_next_year
0,AFG,1991,NaN,NaN,NaN,NaN,48.039,NaN,NaN,7.946,...,8.0,0.0,4.0,1.0,2.0,2.0,4.0,-8.0,1.0,1
1,AFG,1996,NaN,NaN,NaN,NaN,47.856,NaN,NaN,7.941,...,7.0,0.0,0.0,1.0,0.0,1.0,4.0,-7.0,1.0,0
2,AFG,1997,NaN,NaN,NaN,NaN,47.810,NaN,NaN,7.920,...,7.0,0.0,0.0,1.0,1.0,1.0,4.0,-7.0,1.0,0
3,AFG,1998,NaN,NaN,NaN,NaN,47.789,NaN,NaN,7.941,...,7.0,0.0,0.0,1.0,2.0,1.0,4.0,-7.0,1.0,0
4,AFG,1999,NaN,NaN,NaN,NaN,47.788,NaN,NaN,7.953,...,7.0,0.0,0.0,1.0,3.0,1.0,4.0,-7.0,1.0,0


### Imputation

In [62]:
### First try tree model 

### Models